In [1]:
import os
import sys
import numpy as np
import networkx as nx
import pandas as pd
sys.path.append(os.path.join(os.getcwd(), '../', 'data'))
from common_localities import locality_to_common_locality


nodes_df = pd.read_csv("../data/final_metadata.csv", index_col=0)

nodes_df.columns

Index(['nodes_index', ' name', 'most_frequent_locality', 'group',
       'most_frequent_hostgroup'],
      dtype='object')

In [2]:
G = nx.Graph()
for id_parasite, parasite in nodes_df.iterrows():
    G.add_node(
        parasite['nodes_index'],
        name=parasite[' name'],
        most_frequent_locality=parasite['most_frequent_locality'],
        most_frequent_parasite_group=parasite['group'],
        animals_group=parasite['most_frequent_hostgroup']
    )
    
# two steps:
# first create nods with attributes only and then connect them
 
G.number_of_nodes()

18643

In [6]:
import pickle
import os

# Directory containing the CSV files
directory = '../data/trimmed_network_projections'

# Dictionary to store dataframes
dataframes = {}

# Iterate through the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        dataframes[filename] = df

# Print the names of the loaded dataframes
print("Loaded dataframes:", list(dataframes.keys()))


# Create a copy of graph G

# Iterate through each dataframe in the dictionary
for filename, df in dataframes.items():
    G_copy = G.copy()
    # Iterate through each row in the dataframe
    for _, row in df.iterrows():
        src = row['src']
        trg = row['trg']
        nij = row['nij']
        score = row['score']
        # Add edge if src and trg are not the same (no self loop)
        if src != trg:
            G_copy.add_edge(src, trg,attr=[nij,score])

    # Print the number of edges added
    print("Number of edges added:", G_copy.number_of_edges(), G_copy.number_of_nodes())
    name = filename.split('.')[0]
    # print(G)
    with open('../data/trimmed_networks_pkl/'+name+'.pkl', 'wb') as f:
        pickle.dump(G_copy, f)


Loaded dataframes: ['1_resource_allocation_naive.csv', '1_simple_disparity_filter.csv', '2_resource_allocation_naive.csv', '2_simple_disparity_filter.csv']
Number of edges added: 15588 18643
Number of edges added: 443701 18643
Number of edges added: 42691 18643
Number of edges added: 336278 18643
